In [10]:
# Movie Summary Generator from transformer architecture. Using Wikipedia movie summaries from Kaggle.

# First read in the entire dataset



TypeError: object of type '_csv.reader' has no len()